In [1]:
import tensorflow_hub as hub 
import tensorflow as tf
import numpy as np 
import pandas as pd 
from collections import Counter
from tqdm.autonotebook import tqdm 

import warnings
warnings.filterwarnings("ignore")

/home/chen/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from utils import *

In [3]:
import re 
import spacy
import nltk 
from nltk.corpus import stopwords

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

# 1. 准备数据

In [12]:
# 数据清洗
def clean_text(text):
    # 移除url
    text = re.sub(r"http\S+","", text)
    # 移除标点符号
    punctuation = "''#$%^&*+-<>=:/@{}|~\\/;()`,."
    #text = "".join([ch for ch in text if ch not in set(punctuation)])
    text = text.lower()
    text = text.replace(r"[0-9]", "")
    # 对单词进行标准化
    #nlp = spacy.load("en", diable=["parser", "ner"])
    #text = " ".join([token.lemma_ for token in nlp(text)])
    
    stoplist = stopwords.words("english")
    text = " ".join([token for token in text.split() if token.strip() not in stoplist and token.strip() not in set(punctuation)])
    # 移除空格
    text = " ".join(text.split())
    # 超过200的只取前200个
    text = " ".join(text.split()[:200])
    return text

In [5]:
# 设置基础的配置
class Config(dict):
    def __init__(self):
        super().__init__()
        
        #定义训练参数
        self['num_epochs'] = 1 
        self['batch_size'] = 32 
        self['sequenceLength'] = 200
        self['evaluateEvery'] = 200 
        self['checkpointEvery'] = 200 
        
        # 学习率衰减
        self['learningRate'] = 0.01 
        self['decay_steps'] = 100
        self['decay_rate'] = 0.9 
        self['grad_clip'] = 4.0 
        
        # 定义模型参数
        self['embeddingSize'] = 1024
        self['dropoutProb'] = 0.5
        self['l2RegLambda'] = 0.001
        self['hiddenSizes'] = [128]
        
        # 设置基础参数
        self['numClasses'] = 1
        self['train_size'] = 0.8 
        self.threshold = 0.5 
        
        # 保存模型的参数
        self['checkpoint_dir'] = "../model/ELMO/imdb/checkpoint"
        self['summary_dir'] = "../model/ELMO/imdb/summary"
        self['max_to_keep'] = 5
        

# 2 定义模型类和训练类

## 2.1 定义模型类

In [6]:
class BiLSTMAttention(BaseModel):
    def __init__(self, config):
        super().__init__(config)
        self.elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        self.build_model()
        self.init_saver()
        
    def build_model(self):
        # 输入层
        self.inputX = tf.placeholder(tf.string, [None], name="inputX")
        #self.inputL = tf.placeholder(tf.int32, [None], name="inputL")
        self.inputY = tf.placeholder(tf.float32, [None], name="inputY")
        
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # 输出 [batch, seq_len, 1024]
        elmo_result = self.elmo(self.inputX, signature='default', as_dict=True)
        self.embededWords = elmo_result["elmo"]
        
        ## 如果序列的最大长度小于200，则剩余部分用全0向量填充
        ## 如果长度不足则填充
        #if current_len < self.config['sequenceLength']:
        #self.embededWords = tf.pad(self.embededWords, 
        #                               paddings=[[0,0],[0, self.config["sequenceLength"]-current_len], [0, 0]])
            
        
        # 定义双向LSTM模型
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(self.config['hiddenSizes']):
                with tf.name_scope(f"Layer_{idx}"):
                    ## 定义前向的LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize,
                                                                                  state_is_tuple=True),
                                                              output_keep_prob=self.dropout_keep_prob)
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize,
                                                                                  state_is_tuple=True),
                                                              output_keep_prob=self.dropout_keep_prob)
                    
                    # 采用动态RNN
                    outputs_, current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell,
                                                                             self.embededWords,
                                                                             dtype=tf.float32,
                                                                             scope=f"bi-lstm_{idx}")
                    # 对outputs的fw和bw结果进行拼接
                    ## [batch_size, seq_len, hidden_size*2]
                    self.embededWords = tf.concat(outputs_, 2)
                    
        ## 按照最后一个维度进行切分
        outputs  = tf.split(self.embededWords, 2, -1)
        with tf.name_scope("Attention"):
            ## 维度是 [batch, seq_len, hidden_size]
            H = outputs[0] + outputs[1]
            ## 得到Attention的输出
            output = self._attention(H)
            outputSize = self.config['hiddenSizes'][-1]
            
        # 输出层
        with tf.name_scope("output"):
            self.logits = tf.layers.dense(output, self.config['numClasses'],
                                         kernel_initializer=tf.initializers.glorot_normal(),
                                         bias_initializer=tf.initializers.constant(0.1))
            self.predictions = tf.nn.sigmoid(self.logits)
            
        # 损失的计算
        l2_loss = tf.constant(0.0)
        with tf.name_scope("loss"):
            losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(tf.reshape(self.inputY, [-1, 1]),
                                                                           dtype=tf.float32),
                                                            logits=self.logits)
            self.loss = tf.reduce_mean(losses)
            if self.config['l2RegLambda']:
                with tf.variable_scope("dense", reuse=True):
                    outputW = tf.get_variable("kernel")
                    l2_loss += tf.nn.l2_loss(outputW)
                    
            self.loss += self.config['l2RegLambda'] * l2_loss
        
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            learning_rate = tf.train.exponential_decay(self.config['learningRate'],
                                                      self.global_step_tensor,
                                                      self.config['decay_steps'],
                                                      self.config['decay_rate'],
                                                      staircase=True)
            ## 使用梯度削减防止梯度爆炸
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(self.loss)
            for idx, (grad, var) in enumerate(grads_and_vars):
                if grad is not None:
                    grads_and_vars[idx] = (tf.clip_by_norm(grad, self.config['grad_clip']), var)
                    
            self.train_op = optimizer.apply_gradients(grads_and_vars, global_step=self.global_step_tensor)
            
    # 定义注意力结构
    def _attention(self, H):
        # 获得最后一层LSTM的神经元数量
        hiddenSize = self.config['hiddenSizes'][-1]
        # 初始化key
        key = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1), name="key")
        # 对Bi-LSTM的结果进行激活
        M = tf.tanh(H)
        ## 形状 [batch*seq_len, 1]
        restoreM = tf.tensordot(M, key, axes=((2), (0)))
        #restoreM = tf.squeeze(newM, 2)
        # 用归一化除以得到 [batch, seq_len]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的权重对H进行加权求和
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.expand_dims(self.alpha, 2))
        output = tf.tanh(tf.squeeze(r, 2))
        # 进行dropout处理
        output = tf.nn.dropout(output, self.dropout_keep_prob)
        return output
    
    def init_saver(self):
        self.saver = tf.train.Saver(max_to_keep=self.config['max_to_keep'])

In [7]:
class Trainer(BaseTrain):
    def __init__(self, sess, model, data, config, logger):
        super().__init__(sess, model, data, config, logger)
        self.train = data[0]
        self.eval = data[1]
        
    def train_epoch(self):
        num_iter_per_epoch = self.train.length // self.config['batch_size']
        for i in tqdm(range(num_iter_per_epoch)):
            ## 获取训练结果
            loss, metrics, step = self.train_step()
            train_acc = metrics['accuracy']
            train_f_score = metrics['f_score']
            
            # 将训练过程的损失写入
            summaries_dict = {"loss": loss, 
                             "acc": np.array(train_acc),
                             "f_score": np.array(train_f_score)}
            self.logger.summarize(step, summarizer="train", scope="train_summary",
                                 summaries_dict=summaries_dict)
            if step % self.config['evaluateEvery'] == 0: 
                print("Train —— Step: {} | Loss: {} | Acc: {} : F1_Score: {}".format(
                    step, loss, train_acc, train_f_score))
                # 对测试集进行评估
                eval_losses = []
                eval_pred = []
                eval_true = []
                for batchEval in self.eval.iter_all(self.config['batch_size']):
                    loss, predictions = self.eval_step(batchEval)
                    eval_losses.append(loss)
                    eval_pred.extend(predictions)
                    eval_true.extend(batchEval[-1])
                getMetric = Metric(np.array(eval_pred), np.array(eval_true),
                                  self.config)
                metrics = getMetric.get_metrics()
                eval_prec = np.round(metrics['precision'], 5)
                eval_recall = np.round(metrics['recall'], 5)
                loss_mean = np.round(np.mean(eval_losses), 5)
                print("Evaluation —— Loss: {} | Precision: {} | Recall: {}".format(
                    loss_mean, eval_prec, eval_recall))
                summaries_dict = {"loss": np.array(loss_mean),
                                 "precision": np.array(eval_prec), 
                                 "recall": np.array(eval_recall)}
                self.logger.summarize(step, summarizer="test", scope="test_summary",
                                     summaries_dict=summaries_dict)
            if step % self.config['checkpointEvery'] == 0: 
                self.model.save(self.sess)
            
            
    def train_step(self):
        batch_x,  batch_y = next(self.train.next_batch(self.config['batch_size']))
        feed_dict = {self.model.inputX: batch_x,
                     #self.model.inputL: batch_len, 
                    self.model.inputY: batch_y,
                    self.model.dropout_keep_prob: self.config['dropoutProb']}
    
        _, loss, predictions, step = self.sess.run([self.model.train_op,
                                                   self.model.loss,
                                                   self.model.predictions, 
                                                   self.model.global_step_tensor],
                                                  feed_dict=feed_dict)
        getMetric = Metric(predictions, batch_y, self.config)
        metrics = getMetric.get_metrics()
        return loss, metrics, step
    
    def eval_step(self, *batch):
        feed_dict = {self.model.inputX: batch[0],
                     #self.model.inputL: batch[1],
                    self.model.inputY: batch[-1],
                    self.model.dropout_keep_prob: 1.0}
        loss, predictions = self.sess.run([self.model.loss, self.model.predictions],
                                         feed_dict=feed_dict)
        return loss, predictions

# 3. 训练

In [8]:
imdb = pd.read_csv("../data/imdb/labeldTrain.csv")

In [13]:
imdb['review'] = imdb['review'].apply(clean_text)

#imdb['length'] = imdb['review'].apply(lambda s: len(s.split()))

In [136]:
def pad_inputs(reviews, length, config):
    outputs = []
    for i in range(len(reviews)):
        review = reviews[i].split() + [""] * (config['sequenceLength'] - length[i])
        outputs.append(review)
    return np.array(outputs)

In [142]:
class DataGenerator:
    def __init__(self, y, *x):
        self.x = x
        self.y = y
        self.length = len(y)
        ## 计算不同类别的比例
        unique = Counter(self.y.ravel())
        self.ratio = [(key, value / self.length) for key, value in unique.items()]
        self.indices = []
        for key, _ in self.ratio:
            index = np.where(y.ravel() == key)
            self.indices.append(index)

    def next_batch(self, batch_size):
        '''
        生成每一个batch的数据集
        '''
        choose = np.array([])
        for i in range(len(self.indices)):
            idx = np.random.choice(self.indices[i][0],
                                   max(1, min(len(self.indices[i][0]), int(batch_size * self.ratio[i][1]))))
            choose = np.append(choose, idx)
        choose = np.random.permutation(choose).astype("int64")

        result = []
        for item in self.x:
            result.append(item[choose])
        result.append(self.y[choose])
        yield result

    def iter_all(self, batch_size):
        '''
        按照batch迭代所有数据
        '''
        numBatches = self.length // batch_size + 1
        for i in range(numBatches):
            result = []
            start = i * batch_size
            end = min(start + batch_size, self.length)
            for item in self.x:
                result.append(np.array(item[start:end]))
            batchY = np.array(self.y[start:end], dtype="float32")
            result.append(batchY)
            yield result

In [14]:
def main():
    config = Config()
    reviews = imdb['review'].values
    labels = imdb['sentiment'].values
    #length = imdb['length'].values
    
    #reviews = pad_inputs(reviews, length, config)
    ## 表示总的数量
    nums = imdb.shape[0]
    
    
    train_idx = slice(0, int(nums * config['train_size']))
    val_idx = slice(int(nums*config['train_size']), nums)
    
    train_X,  train_y, eval_X,  eval_y = reviews[train_idx],   labels[train_idx], reviews[val_idx],  labels[val_idx]
    train_data = DataGenerator(train_y, train_X)
    eval_data = DataGenerator(eval_y, eval_X)
    pack_data = [train_data, eval_data]
    
    tf.reset_default_graph()
    
    ## 设置计算图的配置
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9 
    
    sess = tf.Session(config=session_conf)
    
    # 创建模型
    model = BiLSTMAttention(config)
    logger = Logger(sess, config)
    
    trainer = Trainer(sess, model, pack_data, config, logger)
    trainer.train_all()